In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder.master("local[5]").enableHiveSupport().getOrCreate()

25/06/24 06:11:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/24 06:11:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/06/24 06:11:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
25/06/24 06:11:18 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
25/06/24 06:11:18 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
25/06/24 06:11:18 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
25/06/24 06:11:18 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
25/06/24 06:11:18 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
25/06/24 06:11:18 WARN Utils: Serv

In [30]:
ecommerce = spark.read.csv("ecommerce/ecommerce.csv", header=True, sep=',', inferSchema=True)

In [31]:
ecommerce.printSchema()

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- InvoiceDate: string (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)



In [64]:
ecommerce = ecommerce.withColumn("InvoiceDate", ecommerce.InvoiceDate.cast(DateType()))

In [65]:
ecommerce = ecommerce.withColumn("total_amount", ecommerce.Quantity*ecommerce.UnitPrice)

In [66]:
ecommerce.printSchema()

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [67]:
ecommerce.describe().show()

+-------+------------------+-----------------+-------------+-----------------+-----------------+-----------------+--------------+------------------+
|summary|         InvoiceNo|        StockCode|  Description|         Quantity|        UnitPrice|       CustomerID|       Country|      total_amount|
+-------+------------------+-----------------+-------------+-----------------+-----------------+-----------------+--------------+------------------+
|  count|              1000|             1000|         1000|             1000|             1000|             1000|          1000|              1000|
|   mean|        551239.811|31507.35562805873|         null|            10.77|49.06149000000005|        54980.499|          null| 519.1824999999998|
| stddev|262835.67138697667|19022.31611502283|         null|5.878746436074587|28.50614703004247|26513.18342163287|          null|441.41107975523306|
|    min|            101024|            21730|    Able item|                1|             1.02|          

In [68]:
null_counts = ecommerce.select([sum(col(c).isNull().cast('int')).alias(c) for c in ecommerce.columns])
null_counts.show()

+---------+---------+-----------+--------+---------+-----------+----------+-------+------------+
|InvoiceNo|StockCode|Description|Quantity|UnitPrice|InvoiceDate|CustomerID|Country|total_amount|
+---------+---------+-----------+--------+---------+-----------+----------+-------+------------+
|        0|        0|          0|       0|        0|          0|         0|      0|           0|
+---------+---------+-----------+--------+---------+-----------+----------+-------+------------+



In [69]:
ecommerce = ecommerce.dropna()

In [70]:
ecommerce.printSchema()

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [71]:
ecommerce.show(10)

+---------+---------+--------------+--------+---------+-----------+----------+--------------+------------------+
|InvoiceNo|StockCode|   Description|Quantity|UnitPrice|InvoiceDate|CustomerID|       Country|      total_amount|
+---------+---------+--------------+--------+---------+-----------+----------+--------------+------------------+
|   620959|    71053|   Budget item|       1|    74.41| 2024-09-22|     99359|         Spain|             74.41|
|   314861|   84029G|     Take item|       5|    73.91| 2024-09-13|     53393|       Germany|369.54999999999995|
|   215905|    22632|     Race item|      14|     4.15| 2024-08-18|     35371|       Germany| 58.10000000000001|
|   938037|   84029G|Beautiful item|       8|    51.03| 2025-01-28|     35508|United Kingdom|            408.24|
|   984993|    22086|    Laugh item|       7|    71.89| 2025-05-31|     58208|         Italy|            503.23|
|   196731|   84029G|   Course item|      15|    59.34| 2023-12-11|     51610|United Kingdom|   

In [72]:
ecommerce.write.mode('overwrite').partitionBy('Country').parquet("hdfs:///user/varunm15t38hedu/ecommerce/parquet")

In [73]:
spark.sql("msck repair table varunmdb.ecom_transactions")

DataFrame[]

In [74]:
df = spark.read.table("varunmdb.ecom_transactions")

In [79]:
spark.sql("select * from varunmdb.ecom_transactions limit 10").show()

+---------+---------+--------------+--------+---------+-----------+----------+------------------+-------+
|invoiceno|stockcode|   description|quantity|unitprice|invoicedate|customerid|      total_amount|country|
+---------+---------+--------------+--------+---------+-----------+----------+------------------+-------+
|   984993|    22086|    Laugh item|       7|    71.89| 2025-05-31|     58208|            503.23|  Italy|
|   251459|   84029G|      Job item|      11|    11.12| 2025-02-26|     56320|            122.32|  Italy|
|   142595|   85123A|  Teacher item|      15|    54.09| 2025-05-15|     15402|            811.35|  Italy|
|   591728|   84029G| Question item|      18|    54.36| 2024-01-11|     96682|            978.48|  Italy|
|   954568|    71053|     Whom item|       5|    63.12| 2024-01-21|     79115|315.59999999999997|  Italy|
|   900457|    22633|Executive item|      18|    17.35| 2025-06-21|     82908|             312.3|  Italy|
|   765243|   84406B|    Still item|      15| 

In [86]:
import happybase

In [88]:
connection = happybase.Connection('master')
table = connection.table('varuntcs:ecom_txn')

In [89]:
for row in ecommerce.limit(10).collect():
    row_key = f"{row.InvoiceNo}_{row.StockCode}"
    table.put(row_key.encode(), {
        b'info:InvoiceNo': str(row.InvoiceNo).encode(),
        b'info:StockCode': str(row.StockCode).encode(),
        b'info:Quantity': str(row.Quantity).encode(),
        b'info:UnitPrice': str(row.UnitPrice).encode(),
        b'info:CustomerID': str(row.CustomerID).encode(),
        b'info:InvoiceDate': str(row.InvoiceDate).encode(),
        b'info:total_amount': str(row.total_amount).encode(),
        b'info:Country': str(row.Country).encode()
    })

In [90]:
sample_key = f"{row.InvoiceNo}_{row.StockCode}".encode()
print(table.row(sample_key))

{b'info:Country': b'Italy', b'info:CustomerID': b'15402', b'info:InvoiceDate': b'2025-05-15', b'info:InvoiceNo': b'142595', b'info:Quantity': b'15', b'info:StockCode': b'85123A', b'info:UnitPrice': b'54.09', b'info:total_amount': b'811.35'}


In [91]:
ecommerce.printSchema()

root
 |-- InvoiceNo: integer (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- InvoiceDate: date (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- total_amount: double (nullable = true)



In [96]:
ecommerce.groupBy("Country").count().alias("count").orderBy(col("count").desc()).show()

+--------------+-----+
|       Country|count|
+--------------+-----+
|         Spain|  137|
|         Italy|  136|
|       Finland|  134|
|        Norway|  128|
|       Germany|  122|
|        France|  117|
|   Netherlands|  115|
|United Kingdom|  111|
+--------------+-----+



In [122]:
ecommerce.select(concat(year(col("InvoiceDate")),lit(" / "),month(col("InvoiceDate"))).alias("month"),col("total_amount")).groupBy("month").agg(round(sum(col("total_amount")),2).alias("total_amount")).orderBy(col("month").desc()).show()

+---------+------------+
|    month|total_amount|
+---------+------------+
| 2025 / 6|    18298.27|
| 2025 / 5|    27481.67|
| 2025 / 4|    35579.92|
| 2025 / 3|    14941.55|
| 2025 / 2|    17796.71|
| 2025 / 1|    19554.19|
| 2024 / 9|    18419.35|
| 2024 / 8|    19718.89|
| 2024 / 7|    12879.05|
| 2024 / 6|    19680.16|
| 2024 / 5|    28597.72|
| 2024 / 4|    21026.57|
| 2024 / 3|    19575.76|
| 2024 / 2|    15686.61|
|2024 / 12|    17126.25|
|2024 / 11|    19705.84|
|2024 / 10|    24134.25|
| 2024 / 1|    25522.79|
| 2023 / 9|    17917.02|
| 2023 / 8|    31184.57|
+---------+------------+
only showing top 20 rows



In [132]:
ecommerce.select(["Description","Quantity"]).groupBy("Description").agg(sum(col("Quantity")).alias("Quantity")).orderBy(col("Quantity").desc()).show(10)

+-------------+--------+
|  Description|Quantity|
+-------------+--------+
|  Travel item|      63|
|    Bill item|      62|
|    Down item|      57|
|    From item|      53|
|   Could item|      53|
|    Cell item|      52|
|Democrat item|      51|
|   Under item|      50|
|   Tough item|      50|
|  Simple item|      50|
+-------------+--------+
only showing top 10 rows



In [136]:
ecommerce.select(round(sum("total_amount"),2).alias("total_amount")).show()

+------------+
|total_amount|
+------------+
|    519182.5|
+------------+



In [145]:
ecommerce.select(col("InvoiceNo"),col("Quantity")).groupBy("InvoiceNo").agg(avg(col("Quantity")).alias("Quantity")).show()

+---------+--------+
|InvoiceNo|Quantity|
+---------+--------+
|   862910|     5.0|
|   638120|     9.0|
|   820904|    20.0|
|   774291|    17.0|
|   767940|    19.0|
|   989559|    11.0|
|   425639|     4.0|
|   574190|    13.0|
|   811049|     4.0|
|   903221|     2.0|
|   581784|     2.0|
|   943924|    20.0|
|   267163|    15.0|
|   847356|    14.0|
|   114166|     1.0|
|   395783|    17.0|
|   838580|    20.0|
|   274021|    15.0|
|   368952|     1.0|
|   514254|    20.0|
+---------+--------+
only showing top 20 rows

